In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
filename = 'datasets/I1-2019.csv'
df = pd.read_csv(filename)

In [3]:
N_PREV_GAMES = 6

In [4]:
df.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY',
       'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH',
       'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD',
       'VCA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA',
       'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh',
       'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'PSCH', 'PSCD', 'PSCA'],
      dtype='object')

In [11]:
teams = df.HomeTeam.unique().tolist()
print(teams)

['Chievo', 'Lazio', 'Bologna', 'Empoli', 'Parma', 'Sassuolo', 'Torino', 'Atalanta', 'Juventus', 'Napoli', 'Cagliari', 'Fiorentina', 'Frosinone', 'Genoa', 'Inter', 'Spal', 'Udinese', 'Roma', 'Milan', 'Sampdoria']


In [118]:
stats = df[['Date','HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']].copy()

In [119]:
stats.loc[:,'HomeRating'] = 0
stats.loc[:,'AwayRating'] = 0

In [120]:
def scored(row, team):
    if row['HomeTeam'] == team:
        return row.FTHG
    if row['AwayTeam'] == team:
        return row.FTAG
    
def conceded(row, team):
    if row['HomeTeam'] == team:
        return row.FTAG
    if row['AwayTeam'] == team:
        return row.FTHG

def get_last_scored(df, team):
    tmpdf = df[(df.HomeTeam == team)|(df.AwayTeam == team)].copy()
    tmpdf['scored'] = tmpdf.apply(scored, team=team, axis=1)
    scored6 = tmpdf.scored.sum()
    return scored6

print(get_last_scored(stats.iloc[0:60,:], 'Juventus'))

def get_last_conceded(df, team):
    tmpdf = df[(df.HomeTeam == team)|(df.AwayTeam == team)].copy()
    tmpdf['conceded'] = tmpdf.apply(conceded, team=team, axis=1)
    conceded6 = tmpdf.conceded.sum()
    return conceded6

print(get_last_conceded(stats.iloc[0:60,:], 'Juventus'))

13
4


In [137]:
for i in range(60,stats.shape[0]):
    start = i - 10*N_PREV_GAMES # 0
    end = start + 10*N_PREV_GAMES # 60
    
    # HomeRating
    stats.iloc[i,6] = get_last_scored(stats.iloc[start:end,:], stats.iloc[end].HomeTeam) - \
                        get_last_conceded(stats.iloc[start:end,:], stats.iloc[end].HomeTeam)
    # AwayRating
    stats.iloc[i,7] = get_last_scored(stats.iloc[start:end,:], stats.iloc[end].AwayTeam) - \
                        get_last_conceded(stats.iloc[start:end,:], stats.iloc[end].AwayTeam)

In [140]:
stats[59:].head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HomeRating,AwayRating
59,29/09/2018,Inter,Cagliari,2,0,H,0,0
60,29/09/2018,Juventus,Napoli,3,1,H,9,5
61,29/09/2018,Roma,Lazio,3,1,H,2,3
62,30/09/2018,Bologna,Udinese,2,1,H,-5,1
63,30/09/2018,Chievo,Torino,0,1,A,-9,-2


In [141]:
get_last_scored(stats.iloc[0:60,:], 'Napoli')

12

In [142]:
get_last_conceded(stats.iloc[0:60,:], 'Napoli')

7

In [147]:
df  = stats.copy()
#team = 'Napoli'
#df[(df.HomeTeam == team)|(df.AwayTeam == team)]
df[60:]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HomeRating,AwayRating
60,29/09/2018,Juventus,Napoli,3,1,H,9,5
61,29/09/2018,Roma,Lazio,3,1,H,2,3
62,30/09/2018,Bologna,Udinese,2,1,H,-5,1
63,30/09/2018,Chievo,Torino,0,1,A,-9,-2
64,30/09/2018,Fiorentina,Atalanta,2,0,H,7,1
65,30/09/2018,Frosinone,Genoa,1,2,A,-16,-1
66,30/09/2018,Parma,Empoli,1,0,H,-2,-4
67,30/09/2018,Sassuolo,Milan,1,4,A,5,0
68,01/10/2018,Sampdoria,Spal,2,1,H,6,-3
69,05/10/2018,Torino,Frosinone,3,2,H,0,-13
